# Importing libraries and downloading packages

In [1]:
import nltk
import numpy as np

In [2]:
# downloading model to tokenize message
nltk.download('punkt')
# downloading stopwords- not want these words to take up space in database or processing time
nltk.download('stopwords')
# downloading wordnet, which contains all lemmas of english language
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kanika\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kanika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kanika\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from nltk.stem import WordNetLemmatizer


#Lemmatization- the process of grouping together the different forms of a word to a single item.

# Function to clean text

In [4]:
def clean_corpus(corpus):
  # splitting words - lowering every word in text and handling punctuation
  corpus = [ doc.lower() for doc in corpus]
  cleaned_corpus = []
  
  stop_words = stopwords.words('english')
  wordnet_lemmatizer = WordNetLemmatizer()

  # iterating over every text
  # Iteration- process of looping through the objects ina collection
  for doc in corpus:
    # tokenizing text
    # simplifies the words
    tokens = word_tokenize(doc)
    cleaned_sentence = [] 
    for token in tokens: 
      # removing stopwords, and punctuation
      if token not in stop_words and token.isalpha(): 
        # applying lemmatization
        cleaned_sentence.append(wordnet_lemmatizer.lemmatize(token)) 
    cleaned_corpus.append(' '.join(cleaned_sentence))
  return cleaned_corpus

# Loading and cleaning intents

In [5]:
import json
with open('intents.json') as file:
    intents = json.load(file)

In [6]:
corpus = []
tags = []
for intent in intents['intents']:
    # taking all patterns in intents to train a neural network
    for pattern in intent['patterns']:
        corpus.append(pattern)
        tags.append(intent['tag'])

In [7]:
#sentences without the unnecessary words
cleaned_corpus = clean_corpus(corpus)

# Vectorizing intents

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cleaned_corpus)

In [ ]:
X.shape

(38, 40)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(tags).reshape(-1,1))

# Training neural network

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
                    Dense(128, input_shape=(X.shape[1],), activation='relu'),
                    Dropout(0.2),
                    Dense(64, activation='relu'),
                    Dropout(0.2),
                    Dense(y.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 128)               5248      
_________________________________________________________________
dropout_18 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_19 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 8)                 520       
Total params: 14,024
Trainable params: 14,024
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X.toarray(), y.toarray(), epochs=20, batch_size=1)

Epoch 1/20
38/38 [==============================] - 0s 1ms/step - loss: 2.0407 - accuracy: 0.1316 
Epoch 2/20
38/38 [==============================] - 0s 778us/step - loss: 1.8970 - accuracy: 0.4474
Epoch 3/20
38/38 [==============================] - 0s 875us/step - loss: 1.7468 - accuracy: 0.6053
Epoch 4/20
38/38 [==============================] - 0s 1ms/step - loss: 1.5789 - accuracy: 0.6316
Epoch 5/20
38/38 [==============================] - 0s 1ms/step - loss: 1.3890 - accuracy: 0.7895
Epoch 6/20
38/38 [==============================] - 0s 954us/step - loss: 1.2050 - accuracy: 0.8684
Epoch 7/20
38/38 [==============================] - 0s 1ms/step - loss: 1.0422 - accuracy: 0.7895
Epoch 8/20
38/38 [==============================] - 0s 1ms/step - loss: 0.8123 - accuracy: 0.9211
Epoch 9/20
38/38 [==============================] - 0s 889us/step - loss: 0.6547 - accuracy: 0.8684
Epoch 10/20
38/38 [==============================] - 0s 790us/step - loss: 0.5608 - accuracy: 0.9474
Epoch 11

# Classifying messages to intent

In [ ]:
# if prediction for every tag is low, then we want to classify that message as noanswer
INTENT_NOT_FOUND_THRESHOLD = 0.40

def predict_intent_tag(message):
  message = clean_corpus([message])
  X_test = vectorizer.transform(message)
  y = model.predict(X_test.toarray())
  # if probability of all intent is low, classify it as noanswer
  if y.max() < INTENT_NOT_FOUND_THRESHOLD:
    return 'noanswer'
  
  prediction = np.zeros_like(y[0])
  prediction[y.argmax()] = 1
  tag = encoder.inverse_transform([prediction])[0][0]
  return tag




In [ ]:
import random


In [ ]:
def get_intent(tag):
  # to return complete intent from intent tag
  for intent in intents['intents']:
    if intent['tag'] == tag:
      return intent

# Complete chat bot

In [ ]:
while True:
  # get message from user
  message = input('You: ')
  # predict intent tag using trained neural network
  tag = predict_intent_tag(message)
  # get complete intent from intent tag
  intent = get_intent(tag)
  # generate random response from intent
  response = random.choice(intent['responses'])
  print('Bot: ', response)

  # break loop if intent was goodbye
  if tag == 'goodbye':
    break

Bot:  Hi there, how can I help?
Bot:  Not sure if I understood
Bot:  Hi there, how can I help?
Bot:  
Bot:  the weather conditions prevailing in an area in general or over a long period
Bot:  Not sure if I understood
Bot:  Have a nice day
